# CSP 1

Consider the following map of the counties of Ireland. The goal is to assign colours to each region so that no neighbouring regions have the same colour. 

There are 32 countries and there are 7 colours: Blue, Green, Red, Yellow, Purple, Brown and White 


In [55]:
from typing import Generic, TypeVar, Dict, List, Optional
from abc import ABC, abstractmethod

V = TypeVar('V') # variable type
D = TypeVar('D') # domain type

# Base class for all constraints
class Constraint(Generic[V, D], ABC):
    # The variables that the constraint is between
    def __init__(self, variables: List[V]) -> None:
        self.variables = variables

    # Must be overridden by subclasses
    @abstractmethod
    def satisfied(self, assignment: Dict[V, D]) -> bool:
        ...
        
class CSP(Generic[V, D]):
    def __init__(self, variables: List[V], domains: Dict[V, List[D]]) -> None:
        self.variables: List[V] = variables # variables to be constrained
        self.domains: Dict[V, List[D]] = domains # domain of each variable
        self.constraints: Dict[V, List[Constraint[V, D]]] = {}
        for variable in self.variables:
            self.constraints[variable] = []
            if variable not in self.domains:
                raise LookupError("Every variable should have a domain assigned to it.")

    def add_constraint(self, constraint: Constraint[V, D]) -> None:
        for variable in constraint.variables:
            if variable not in self.variables:
                raise LookupError("Variable in constraint not in CSP")
            else:
                self.constraints[variable].append(constraint)

    # Check if the value assignment is consistent by checking all constraints
    # for the given variable against it
    def consistent(self, variable: V, assignment: Dict[V, D]) -> bool:
        for constraint in self.constraints[variable]:
            if not constraint.satisfied(assignment):
                return False
        return True

    def backtracking_search(self, assignment: Dict[V, D] = {}) -> Optional[Dict[V, D]]:
        # assignment is complete if every variable is assigned (our base case)
        if len(assignment) == len(self.variables):
            return assignment

        # get all variables in the CSP but not in the assignment
        unassigned: List[V] = [v for v in self.variables if v not in assignment]

        # get the every possible domain value of the first unassigned variable
        first: V = unassigned[0]
        for value in self.domains[first]:
            local_assignment = assignment.copy()
            local_assignment[first] = value
            # if we're still consistent, we recurse (continue)
            if self.consistent(first, local_assignment):
                result: Optional[Dict[V, D]] = self.backtracking_search(local_assignment)
                # if we didn't find the result, we will end up backtracking
                if result is not None:
                    return result
        return None
    
class MapColoringConstraint(Constraint[str, str]):
    def __init__(self, place1: str, place2: str) -> None:
        super().__init__([place1, place2])
        self.place1: str = place1
        self.place2: str = place2

    def satisfied(self, assignment: Dict[str, str]) -> bool:
        # If either place is not in the assignment then it is not
        # yet possible for their colors to be conflicting
        if self.place1 not in assignment or self.place2 not in assignment:
            return True
        # check the color assigned to place1 is not the same as the
        # color assigned to place2
        return assignment[self.place1] != assignment[self.place2]

In [72]:
from typing import Dict, List, Optional
from csp2 import *

In [70]:
if __name__ == "__main__":
    variables: List[str] = ["Kerry", "Cork", "Limerick", "Clare", "Waterford", "Tipperary", "Galway", "Mayo",
                           "Kilkenny","Laois","Offaiy","Roscommon","Stigo","Wexford","Carlow","Kildare","Offaly",
                           "Westmeath","Longford","Leitacim","Wicktow","Dublin","Louth","Meath","Cavan","Monaghan",
                           "Fermanagh","Armagh","Tyrone","Donegal","Down","Antrim","Derry"]
    domains: Dict[str, List[str]] = {}
    for variable in variables:
        domains[variable] = ["Blue", "Green", "Red", "Yellow", "Purple", "Brown", "White"]
    csp: CSP[str, str] = CSP(variables, domains)
    # kerry
    csp.add_constraint(MapColoringConstraint("Kerry", "Cork"))  
    csp.add_constraint(MapColoringConstraint("Kerry", "Limerick"))  
    csp.add_constraint(MapColoringConstraint("Kerry", "Clare"))  
    # Cork
    csp.add_constraint(MapColoringConstraint("Cork", "Kerry")) 
    csp.add_constraint(MapColoringConstraint("Cork", "Limerick"))  
    csp.add_constraint(MapColoringConstraint("Cork", "Clare")) 
    # Limerick
    csp.add_constraint(MapColoringConstraint("Limerick", "Kerry")) 
    csp.add_constraint(MapColoringConstraint("Limerick", "Cork")) 
    csp.add_constraint(MapColoringConstraint("Limerick", "Tipperary")) 
    csp.add_constraint(MapColoringConstraint("Limerick", "Clare")) 
    # Clare
    csp.add_constraint(MapColoringConstraint("Clare", "Kerry"))
    csp.add_constraint(MapColoringConstraint("Clare", "Limerick"))
    csp.add_constraint(MapColoringConstraint("Clare", "Tipperary"))
    csp.add_constraint(MapColoringConstraint("Clare", "Galway"))
    # Waterford
    csp.add_constraint(MapColoringConstraint("Waterford","Cork"))
    csp.add_constraint(MapColoringConstraint("Waterford","Tipperary"))
    csp.add_constraint(MapColoringConstraint("Waterford","Kilkenny"))
    csp.add_constraint(MapColoringConstraint("Waterford","Wexford"))
    # Tipperary
    csp.add_constraint(MapColoringConstraint("Tipperary", "Cork"))
    csp.add_constraint(MapColoringConstraint("Tipperary", "Waterford"))
    csp.add_constraint(MapColoringConstraint("Tipperary", "Kilkenny"))
    csp.add_constraint(MapColoringConstraint("Tipperary", "Laois"))
    csp.add_constraint(MapColoringConstraint("Tipperary", "Offaly"))
    csp.add_constraint(MapColoringConstraint("Tipperary", "Galway"))
    csp.add_constraint(MapColoringConstraint("Tipperary", "Clare"))
    csp.add_constraint(MapColoringConstraint("Tipperary", "Limerick"))
    # Galway
    csp.add_constraint(MapColoringConstraint("Galway", "Clare"))
    csp.add_constraint(MapColoringConstraint("Galway", "Tipperary"))
    csp.add_constraint(MapColoringConstraint("Galway", "Offaly"))
    csp.add_constraint(MapColoringConstraint("Galway", "Roscommon"))
    csp.add_constraint(MapColoringConstraint("Galway", "Mayo"))
    # Mayo
    csp.add_constraint(MapColoringConstraint("Mayo", "Galway"))
    csp.add_constraint(MapColoringConstraint("Mayo", "Roscommon"))
    csp.add_constraint(MapColoringConstraint("Mayo", "Stigo"))
    # Stigo
    csp.add_constraint(MapColoringConstraint("Stigo", "Mayo"))
    csp.add_constraint(MapColoringConstraint("Stigo", "Roscommon"))
    csp.add_constraint(MapColoringConstraint("Stigo", "Leitacim"))
    # Roscommon
    csp.add_constraint(MapColoringConstraint("Roscommon", "Galway"))
    csp.add_constraint(MapColoringConstraint("Roscommon", "Offaly"))
    csp.add_constraint(MapColoringConstraint("Roscommon", "Westmeath"))
    csp.add_constraint(MapColoringConstraint("Roscommon", "Longford"))
    csp.add_constraint(MapColoringConstraint("Roscommon", "Leitacim"))
    csp.add_constraint(MapColoringConstraint("Roscommon", "Stigo"))
    csp.add_constraint(MapColoringConstraint("Roscommon", "Mayo"))
    # Offaly
    csp.add_constraint(MapColoringConstraint("Offaly", "Galway"))
    csp.add_constraint(MapColoringConstraint("Offaly", "Tipperary"))
    csp.add_constraint(MapColoringConstraint("Offaly", "Laois"))
    csp.add_constraint(MapColoringConstraint("Offaly", "Kildare"))
    csp.add_constraint(MapColoringConstraint("Offaly", "Meath"))
    csp.add_constraint(MapColoringConstraint("Offaly", "Westmeath"))
    csp.add_constraint(MapColoringConstraint("Offaly", "Roscommon"))
    # Laois
    csp.add_constraint(MapColoringConstraint("Laois", "Tipperary"))
    csp.add_constraint(MapColoringConstraint("Laois", "Kilkenny"))
    csp.add_constraint(MapColoringConstraint("Laois", "Carlow"))
    csp.add_constraint(MapColoringConstraint("Laois", "Kildare"))
    csp.add_constraint(MapColoringConstraint("Laois", "Offaly"))
    # Kilkenny
    csp.add_constraint(MapColoringConstraint("Kilkenny", "Waterford"))
    csp.add_constraint(MapColoringConstraint("Kilkenny", "Wexford"))
    csp.add_constraint(MapColoringConstraint("Kilkenny", "Carlow"))
    csp.add_constraint(MapColoringConstraint("Kilkenny", "Laois"))
    csp.add_constraint(MapColoringConstraint("Kilkenny", "Tipperary"))
    # Wexford
    csp.add_constraint(MapColoringConstraint("Wexford", "Kilkenny"))
    csp.add_constraint(MapColoringConstraint("Wexford", "Waterford"))
    csp.add_constraint(MapColoringConstraint("Wexford", "Carlow"))
    csp.add_constraint(MapColoringConstraint("Wexford", "Wicktow"))
    # Carlow
    csp.add_constraint(MapColoringConstraint("Carlow","Wexford"))
    csp.add_constraint(MapColoringConstraint("Carlow","Kilkenny"))
    csp.add_constraint(MapColoringConstraint("Carlow","Laois"))
    csp.add_constraint(MapColoringConstraint("Carlow","Kildare"))
    csp.add_constraint(MapColoringConstraint("Carlow","Wicktow"))
    # Wicktow
    csp.add_constraint(MapColoringConstraint("Wicktow", "Wexford"))
    csp.add_constraint(MapColoringConstraint("Wicktow", "Carlow"))
    csp.add_constraint(MapColoringConstraint("Wicktow", "Kildare"))
    csp.add_constraint(MapColoringConstraint("Wicktow", "Dublin"))
    # Dublin
    csp.add_constraint(MapColoringConstraint("Dublin","Wicktow"))
    csp.add_constraint(MapColoringConstraint("Dublin","Kildare"))
    csp.add_constraint(MapColoringConstraint("Dublin","Meath"))
    # Kildare
    csp.add_constraint(MapColoringConstraint("Kildare", "Dublin"))
    csp.add_constraint(MapColoringConstraint("Kildare", "Wicktow"))
    csp.add_constraint(MapColoringConstraint("Kildare", "Meath"))
    csp.add_constraint(MapColoringConstraint("Kildare", "Carlow"))
    csp.add_constraint(MapColoringConstraint("Kildare", "Laois"))
    csp.add_constraint(MapColoringConstraint("Kildare", "Offaly"))
    # Meath
    csp.add_constraint(MapColoringConstraint("Meath", "Dublin"))
    csp.add_constraint(MapColoringConstraint("Meath", "Louth"))
    csp.add_constraint(MapColoringConstraint("Meath", "Monaghan"))
    csp.add_constraint(MapColoringConstraint("Meath", "Cavan"))
    csp.add_constraint(MapColoringConstraint("Meath", "Westmeath"))
    csp.add_constraint(MapColoringConstraint("Meath", "Offaly"))
    csp.add_constraint(MapColoringConstraint("Meath", "Kildare"))
    # Louth
    csp.add_constraint(MapColoringConstraint("Louth", "Meath"))
    csp.add_constraint(MapColoringConstraint("Louth", "Monaghan"))
    csp.add_constraint(MapColoringConstraint("Louth", "Armagh"))
    # Cavan
    csp.add_constraint(MapColoringConstraint("Cavan","Meath"))
    csp.add_constraint(MapColoringConstraint("Cavan","Monaghan"))
    csp.add_constraint(MapColoringConstraint("Cavan","Fermanagh"))
    csp.add_constraint(MapColoringConstraint("Cavan","Leitacim"))
    csp.add_constraint(MapColoringConstraint("Cavan","Longford"))
    csp.add_constraint(MapColoringConstraint("Cavan","Westmeath"))
    # Westmeath
    csp.add_constraint(MapColoringConstraint("Westmeath", "Offaly"))
    csp.add_constraint(MapColoringConstraint("Westmeath", "Meath"))
    csp.add_constraint(MapColoringConstraint("Westmeath", "Cavan"))
    csp.add_constraint(MapColoringConstraint("Westmeath", "Longford"))
    csp.add_constraint(MapColoringConstraint("Westmeath", "Roscommon"))
    # Longford
    csp.add_constraint(MapColoringConstraint("Longford", "Cavan"))
    csp.add_constraint(MapColoringConstraint("Longford", "Westmeath"))
    csp.add_constraint(MapColoringConstraint("Longford", "Roscommon"))
    csp.add_constraint(MapColoringConstraint("Longford", "Leitacim"))
    # Leitacim
    csp.add_constraint(MapColoringConstraint("Leitacim", "Stigo"))
    csp.add_constraint(MapColoringConstraint("Leitacim", "Roscommon"))
    csp.add_constraint(MapColoringConstraint("Leitacim", "Longford"))
    csp.add_constraint(MapColoringConstraint("Leitacim", "Cavan"))
    csp.add_constraint(MapColoringConstraint("Leitacim", "Fermanagh"))
    # Monaghan
    csp.add_constraint(MapColoringConstraint("Monaghan", "Louth"))
    csp.add_constraint(MapColoringConstraint("Monaghan", "Meath"))
    csp.add_constraint(MapColoringConstraint("Monaghan", "Fermanagh"))
    csp.add_constraint(MapColoringConstraint("Monaghan", "Tyrone"))
    csp.add_constraint(MapColoringConstraint("Monaghan", "Armagh"))
    csp.add_constraint(MapColoringConstraint("Monaghan", "Cavan"))
    # Fermanagh
    csp.add_constraint(MapColoringConstraint("Fermanagh", "Leitacim"))
    csp.add_constraint(MapColoringConstraint("Fermanagh", "Cavan"))
    csp.add_constraint(MapColoringConstraint("Fermanagh", "Monaghan"))
    csp.add_constraint(MapColoringConstraint("Fermanagh", "Tyrone"))
    csp.add_constraint(MapColoringConstraint("Fermanagh", "Donegal"))
    # Armagh
    csp.add_constraint(MapColoringConstraint("Armagh", "Louth"))
    csp.add_constraint(MapColoringConstraint("Armagh", "Monaghan"))
    csp.add_constraint(MapColoringConstraint("Armagh", "Down"))
    csp.add_constraint(MapColoringConstraint("Armagh", "Tyrone"))
    # Down
    csp.add_constraint(MapColoringConstraint("Down", "Louth"))
    csp.add_constraint(MapColoringConstraint("Down", "Armagh"))
    csp.add_constraint(MapColoringConstraint("Down", "Antrim"))
    # Antrim
    csp.add_constraint(MapColoringConstraint("Antrim", "Down"))
    csp.add_constraint(MapColoringConstraint("Antrim", "Derry"))
    # Derry
    csp.add_constraint(MapColoringConstraint("Derry", "Antrim"))
    csp.add_constraint(MapColoringConstraint("Derry", "Tyrone"))
    csp.add_constraint(MapColoringConstraint("Derry", "Donegal"))
    # Donegal
    csp.add_constraint(MapColoringConstraint("Donegal", "Derry"))
    csp.add_constraint(MapColoringConstraint("Donegal", "Tyrone"))
    csp.add_constraint(MapColoringConstraint("Donegal", "Fermanagh"))
    # Tyrone
    csp.add_constraint(MapColoringConstraint("Tyrone", "Armagh"))
    csp.add_constraint(MapColoringConstraint("Tyrone", "Fermanagh"))
    csp.add_constraint(MapColoringConstraint("Tyrone", "Donegal"))
    csp.add_constraint(MapColoringConstraint("Tyrone", "Derry"))
    csp.add_constraint(MapColoringConstraint("Tyrone", "Monaghan"))
    
    
    solution: Optional[Dict[str, str]] = csp.backtracking_search()
    if solution is None:
        print("No solution found!")
    else:
        print(solution)

{'Kerry': 'Blue', 'Cork': 'Green', 'Limerick': 'Red', 'Clare': 'Yellow', 'Waterford': 'Blue', 'Tipperary': 'Purple', 'Galway': 'Blue', 'Mayo': 'Green', 'Kilkenny': 'Green', 'Laois': 'Blue', 'Offaiy': 'Blue', 'Roscommon': 'Red', 'Stigo': 'Blue', 'Wexford': 'Red', 'Carlow': 'Yellow', 'Kildare': 'Green', 'Offaly': 'Yellow', 'Westmeath': 'Blue', 'Longford': 'Green', 'Leitacim': 'Yellow', 'Wicktow': 'Blue', 'Dublin': 'Red', 'Louth': 'Blue', 'Meath': 'Purple', 'Cavan': 'Red', 'Monaghan': 'Green', 'Fermanagh': 'Blue', 'Armagh': 'Red', 'Tyrone': 'Yellow', 'Donegal': 'Green', 'Down': 'Green', 'Antrim': 'Blue', 'Derry': 'Red'}


# CSP 2

How can one fill an 8 by 8 table with different numbers so that each number occurs exactly
once in each row and once in each column? 


In [50]:
def main_csp(input_row: int) :
    try :
        curr = 1
        for col in range(input_row):
            urgent = 0
            curr_value = 1
            for row in range(input_row):
                current_value = row + curr
                if current_value > input_row :
                    print(curr_value+urgent, end="   ")
                    urgent += 1
                else :
                    print(current_value, end="   ")
            curr += 1
            print()
    except:
        print("Please Input Number Only !")
        
def main():
    result_value = int(input("Please Input ur Number Cols & Rows : "))
    print("\n")
    print("="*11, "result", "="*10)
    main_csp(result_value)
    print("="*11, "result", "="*10)

In [71]:
from csp2 import main

In [51]:
main()

Please Input ur Number Cols & Rows : 8


=========== result ==========
1   2   3   4   5   6   7   8   
2   3   4   5   6   7   8   1   
3   4   5   6   7   8   1   2   
4   5   6   7   8   1   2   3   
5   6   7   8   1   2   3   4   
6   7   8   1   2   3   4   5   
7   8   1   2   3   4   5   6   
8   1   2   3   4   5   6   7   
=========== result ==========
